In [1]:
import os
import sys
import argparse 
import math
import time
import h5py
import joblib
import subprocess
import numpy as np
import pandas as pd
import scipy
import statsmodels.api as sm
import matplotlib.pyplot as plt 
import seaborn as sns

In [2]:
os.chdir('/Users/pengl7/Downloads/WGS/compare-variants/NIST/')

In [16]:
%ls -lth

total 142592
-rw-r-----  1 pengl7  NIH\Domain Users    15M Oct  1 15:06 long_UpdateAF4_with_title
drwxr-xr-x  9 pengl7  NIH\Domain Users   288B Sep 29 21:38 NIST_ref/
-rw-r--r--@ 1 pengl7  NIH\Domain Users   301B Sep 28 20:55 sum of variants filtering.csv
-rw-r--r--@ 1 pengl7  NIH\Domain Users   9.7K Sep 28 16:43 sum of variants filtering.xlsx
-rw-r--r--  1 pengl7  NIH\Domain Users    20M Sep 28 13:03 long_cleared_QD.csv
-rw-r--r--  1 pengl7  NIH\Domain Users    21M Sep 25 16:58 long_cleared.csv
-rw-r-----  1 pengl7  NIH\Domain Users    12M Sep 25 10:35 long4_with_title


In [5]:
# df = pd.read_csv("long4_with_title", sep="\t",index_col="POS", na_values={}, dtype={'UNMATCH': "category", "DP": "float", "GQ": "float", "MQ": "float"}, )
df = pd.read_csv("long_UpdateAF4_with_title", sep="\t")

/Users/pengl7/opt/anaconda3/envs/genomics/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (6,7,8,9) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
print(df.shape)
print(df.columns.to_list())

(339882, 10)
['POS', 'UNMATCH', 'TYPE', 'QUAL', 'FS', 'AF', 'DP', 'GQ', 'MQ', 'QD']


In [7]:
df.dtypes

POS          int64
UNMATCH      int64
TYPE         int64
QUAL       float64
FS         float64
AF         float64
DP          object
GQ          object
MQ          object
QD          object
dtype: object

In [9]:
df.describe()

,POS,UNMATCH,TYPE,QUAL,FS,AF
count,3.398820e+05,339882.000000,339882.000000,339882.000000,339882.000000,339882.000000
mean,1.256505e+08,0.102515,0.161209,480.435447,1.733839,0.516540
std,7.641421e+07,0.303325,0.367724,720.749342,3.342157,0.293846
min,1.014600e+04,0.000000,0.000000,30.250000,0.000000,0.000000
25%,5.925409e+07,0.000000,0.000000,233.770000,0.000000,0.279553
50%,1.148292e+08,0.000000,0.000000,385.770000,0.000000,0.500000
75%,1.950176e+08,0.000000,0.000000,647.770000,2.218000,0.757788
max,2.489456e+08,1.000000,1.000000,117461.000000,357.189000,1.000000


## Get rid of some non-numeric values and change data types

In [11]:
for col in ['DP', 'GQ', 'MQ', 'QD']:
    print(len(df[df[col]== "."]))

20
20
2
2


In [14]:
myFilter = (df["DP"]!= ".") & (df["GQ"]!= ".") & (df["MQ"]!= ".") & (df["QD"]!= ".")
df = df[myFilter]
print(df.shape)

(339860, 10)


In [20]:
# change data type 
cols = ['DP', 'GQ', 'MQ', "QD"]
for col in cols:
    df[col]= df[col].astype("float")

/Users/pengl7/opt/anaconda3/envs/genomics/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [21]:
df.to_csv("long_cleared_UpdateAF.csv")

## Apply logsistic regression model

In [28]:
df.describe()

,POS,UNMATCH,QUAL,FS,AF,DP,GQ,MQ,QD
count,3.398600e+05,339860.000000,339860.000000,339860.000000,339860.000000,339860.000000,339860.000000,339860.000000,339860.000000
mean,1.256499e+08,0.102457,480.445892,1.733951,0.516533,19.790970,78.836871,59.430544,20.941954
std,7.641397e+07,0.303249,720.769333,3.342236,0.293848,22.881138,27.074520,2.707473,8.644774
min,1.014600e+04,0.000000,30.250000,0.000000,0.000000,0.000000,0.000000,20.000000,0.120000
25%,5.925370e+07,0.000000,233.770000,0.000000,0.279553,14.000000,59.000000,60.000000,13.950000
50%,1.148291e+08,0.000000,385.770000,0.000000,0.500000,19.000000,99.000000,60.000000,20.120000
75%,1.950171e+08,0.000000,647.770000,2.218000,0.757788,24.000000,99.000000,60.000000,28.990000
max,2.489456e+08,1.000000,117461.000000,357.189000,1.000000,3142.000000,99.000000,60.000000,42.890000


In [25]:
df["TYPE"] = df["TYPE"].astype("category")

/Users/pengl7/opt/anaconda3/envs/genomics/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [26]:
this_formula = "UNMATCH ~ TYPE + DP + GQ + QUAL + FS + AF + MQ + QD"
res = sm.formula.glm(formula=this_formula, family=sm.families.Binomial(), data=df).fit() 
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                UNMATCH   No. Observations:               339860
Model:                            GLM   Df Residuals:                   339851
Model Family:                Binomial   Df Model:                            8
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -85342.
Date:                Thu, 01 Oct 2020   Deviance:                   1.7068e+05
Time:                        15:24:49   Pearson chi2:                 3.30e+05
No. Iterations:                     6                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     12.2662      0.120    102.450      0.000      12.032      12.501
TYPE[T.1]      1.5735      0.015    107.630      0.000       1.545       1.602
DP             0.0078      0.001     10.081      0.000       0.006       0.009
GQ            -0.0193      0.000    -69.527      0.000      -0.020      -0.019
QUAL          -0.0002   2.64e-05     -7.791      0.000      -0.000      -0.000
FS             0.0176      0.002      8.302      0.000       0.013       0.022
AF             0.9348      0.027     34.462      0.000       0.882       0.988
MQ            -0.2178      0.002   -110.902      0.000      -0.222      -0.214
QD            -0.0615      0.001    -56.421      0.000      -0.064      -0.059
==============================================================================
"""

In [27]:
print("Coefficeients")
print(res.params)
print()
print("p-Values")
print(res.pvalues)
print()
print("Dependent variables")
print(res.model.endog_names)

Coefficeients
Intercept    12.266173
TYPE[T.1]     1.573524
DP            0.007779
GQ           -0.019305
QUAL         -0.000206
FS            0.017602
AF            0.934848
MQ           -0.217819
QD           -0.061512
dtype: float64

p-Values
Intercept     0.000000e+00
TYPE[T.1]     0.000000e+00
DP            6.735944e-24
GQ            0.000000e+00
QUAL          6.630611e-15
FS            1.025430e-16
AF           2.945804e-260
MQ            0.000000e+00
QD            0.000000e+00
dtype: float64

Dependent variables
UNMATCH


In [29]:
# remove those variants which AF hasn't been updated 
filt1 = (df["AF"]!=0.5) & (df["AF"]!=1.0)
df_MAF = df[filt1]
print(len(df_MAF))

297642


In [30]:

res2 = sm.formula.glm(formula=this_formula, family=sm.families.Binomial(), data=df_MAF).fit() 
res2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                UNMATCH   No. Observations:               297642
Model:                            GLM   Df Residuals:                   297633
Model Family:                Binomial   Df Model:                            8
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -48759.
Date:                Fri, 02 Oct 2020   Deviance:                       97517.
Time:                        11:13:39   Pearson chi2:                 2.90e+05
No. Iterations:                     7                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     12.4685      0.259     48.137      0.000      11.961      12.976
TYPE[T.1]      1.2458      0.022     56.324      0.000       1.202       1.289
DP             0.0024      0.001      2.258      0.024       0.000       0.004
GQ            -0.0206      0.000    -50.653      0.000      -0.021      -0.020
QUAL        8.519e-05   3.83e-05      2.226      0.026    1.02e-05       0.000
FS             0.0154      0.003      5.024      0.000       0.009       0.021
AF            -0.0125      0.037     -0.338      0.735      -0.085       0.060
MQ            -0.2239      0.004    -52.006      0.000      -0.232      -0.215
QD            -0.0463      0.002    -28.104      0.000      -0.050      -0.043
==============================================================================
"""

In [31]:
print("Coefficeients")
print(res2.params)
print()
print("p-Values")
print(res2.pvalues)
print()
print("Dependent variables")
print(res2.model.endog_names)

Coefficeients
Intercept    12.468461
TYPE[T.1]     1.245848
DP            0.002372
GQ           -0.020617
QUAL          0.000085
FS            0.015415
AF           -0.012540
MQ           -0.223882
QD           -0.046327
dtype: float64

p-Values
Intercept     0.000000e+00
TYPE[T.1]     0.000000e+00
DP            2.395434e-02
GQ            0.000000e+00
QUAL          2.599788e-02
FS            5.056474e-07
AF            7.352164e-01
MQ            0.000000e+00
QD           8.727729e-174
dtype: float64

Dependent variables
UNMATCH
